# Modeling and Simulation in Python

Chapter 10 Example: Spiderman

Copyright 2017 Allen Downey

License: [Creative Commons Attribution 4.0 International](https://creativecommons.org/licenses/by/4.0)


In [ ]:
# If you want the figures to appear in the notebook, 
# and you want to interact with them, use
# %matplotlib notebook

# If you want the figures to appear in the notebook, 
# and you don't want to interact with them, use
# %matplotlib inline

# If you want the figures to appear in separate windows, use
# %matplotlib qt5

# tempo switch from one to another, you have to select Kernel->Restart

%matplotlib notebook

from modsim import *


I'll start by getting the units we'll need from Pint.

In [ ]:
m = UNITS.meter
s = UNITS.second
kg = UNITS.kilogram
N = UNITS.newton
degree = UNITS.degree
radian = UNITS.radian

### Spider-Man

In this notebook we'll develop a model of Spider-Man swinging from a springy cable of webbing attached to the top of the Empire State Building.

Initially, Spider-Man is at the top of a nearby builing.  I'll create a `Condition` object to contain the quantities we'll need:

1. According to [the Spider-Man Wiki](http://spiderman.wikia.com/wiki/Peter_Parker_(Earth-616)) Spider-Man weighs 76 kg.

2. Let's assume his terminal velocity is 60 m/s.

3. The length of the web is 100 m.

4. The spring constant of the web is 20 N / m when the cord is stretched, and 0 when it's compressed.

5. The initial angle of the web is 45 degrees to the left of straight down.

In [ ]:
condition = Condition(height = 381 * m,
                      g = 9.8 * m/s**2,
                      mass = 75 * kg,
                      area = 1 * m**2,
                      rho = 1.2 * kg/m**3,
                      v_term = 60 * m / s,
                      duration = 60 * s,
                      length = 100 * m,
                      angle = (270 - 45) * degree,
                      k = 20 * N / m)

Now here's a version of `make_system` that takes a `Condition` object as a parameter.

`make_system` uses the given value of `v_term` to compute the drag coefficient `C_d`.

In [ ]:
def make_system(condition):
    """Makes a System object for the given conditions.
    
    condition: Condition with height, g, mass, diameter, 
               rho, v_term, and duration
    
    returns: System with init, g, mass, rho, C_d, area, and ts
    """
    unpack(condition)
    
    A = Vector(0, height)
    
    theta = angle.to(radian)
    x, y = pol2cart(theta, length)
    L = Vector(x, y)
    P = A + L
    V = Vector(0*m/s, 0*m/s)
    
    init = State(x=P.x, y=P.y, vx=V.x, vy=V.y)
    C_d = 2 * mass * g / (rho * area * v_term**2)
    ts = linspace(0, duration, 501)
    
    
    return System(init=init, g=g, mass=mass, rho=rho,
                  C_d=C_d, area=area, length=length,
                  A=A, k=k, ts=ts)

Let's make a `System`

In [ ]:
system = make_system(condition)
system

Here's the slope function, including acceleration due to gravity and drag.

In [ ]:
def slope_func(state, t, system):
    """Computes derivatives of the state variables.
    
    state: State (x, y, x velocity, y velocity)
    t: time
    system: System object with g, rho, C_d, area, mass
    
    returns: sequence (vx, vy, ax, ay)
    """
    x, y, vx, vy = state
    unpack(system)
    
    a_grav = Vector(0, -g)

    P = Vector(x, y)
    V = Vector(vx, vy)
    L = P - A
    
    f_spring = -k * (L.mag - length) * L.hat()
    a_spring = f_spring / mass
    
    f_drag = -rho * V.mag * V * C_d * area / 2
    a_drag = f_drag / mass
    
    a = a_grav + a_drag + a_spring
    
    return vx, vy, a.x, a.y

As always, let's test the slope function with the initial conditions.

In [ ]:
slope_func(system.init, 0, system)

And then run the simulation.

In [ ]:
run_odeint(system, slope_func)

### Visualizing the results

We can extract the x and y components as `Series` objects.

In [ ]:
xs = system.results.x
ys = system.results.y

The simplest way to visualize the results is to plot x and y as functions of time.

In [ ]:
newfig()
plot(xs, label='x')
plot(ys, label='y')

decorate(xlabel='Time (s)',
         ylabel='Position (m)')

We can plot the velocities the same way.

In [ ]:
vxs = system.results.vx
vys = system.results.vy

In [ ]:
newfig()
plot(vxs, label='vx')
plot(vys, label='vy')

decorate(xlabel='Time (s)',
         ylabel='Velocity (m/s)')

Another way to visualize the results is to plot y versus x.  The result is the trajectory through the plane of motion.

In [ ]:
newfig()
plot(xs, ys, label='trajectory')

decorate(xlabel='x position (m)',
         ylabel='y position (m)')

We can also animate the trajectory.  If there's an error in the simulation, we can sometimes spot it by looking at animations.

In [ ]:
newfig()
decorate(xlabel='x position (m)',
         ylabel='y position (m)',
         xlim=[-105, 105],
         ylim=[0, 381],
         legend=False)

for x, y in zip(xs, ys):
    plot(x, y, 'bo', update=True)
    sleep(0.01)

Here's a function that encapsulates that code and runs the animation in (approximately) real time.

In [ ]:
def animate2d(xs, ys, speedup=1):
    """Animate the results of a projectile simulation.
    
    xs: x position as a function of time
    ys: y position as a function of time
    
    speedup: how much to divide `dt` by
    """
    # get the time intervals between elements
    ts = xs.index
    dts = np.diff(ts)
    dts = np.append(dts, 0)

    # decorate the plot
    newfig()
    decorate(xlabel='x position (m)',
             ylabel='y position (m)',
             xlim=[xs.min(), xs.max()],
             ylim=[ys.min(), ys.max()],
             legend=False)

    # loop through the values
    for x, y, dt in zip(xs, ys, dts):
        plot(x, y, 'bo', update=True)
        sleep(dt / speedup)

In [ ]:
animate2d(system.results.x, system.results.y)